In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import talib as ta
import os
import mplfinance as mpf

In [2]:
def get_time_stamp():
    return datetime.now().strftime('%Y%m%d%H%M%S')

In [3]:
directory = 'bitget-' + get_time_stamp()
os.makedirs(directory, exist_ok=True)

In [4]:
def get_pairs() :
    return requests.get('https://api.bitget.com/api/v2/spot/public/symbols').json()

In [5]:
def get_ticker(symbol, granularity, time_from, time_to) :
    request_str = 'https://api.bitget.com/api/v2/spot/market/candles' + '?symbol=' + symbol + '&granularity=' + granularity + '&startTime=' + str(time_from) + '&endTime=' + str(time_to) + '&limit=1000'
    return requests.get(request_str).json()

In [6]:
width_ratio = 0.2

now = datetime.now()
past = now - timedelta(days=5*365)
time_from_ms = int(past.timestamp() * 1000)
time_to_ms = int(now.timestamp() * 1000)
time_frame = '1Dutc'

In [7]:
pairs = get_pairs()
symbols = [item['symbol'] for item in pairs['data'] if 'symbol' in item and item['symbol'].endswith('USDT')]
remove_list = ['BTCUSDT', 'USDCUSDT', 'USDPUSDT', 'USTCUSDT', 'FDUSDUSDT', 'TUSDUSDT', 'AEURUSDT', 'DAIUSDT', 'USDEUSDT', 'DEUSDUSDT', 'PYUSDUSDT', 'WUSDUSDT', 'WBTCUSDT', 'WETHUSDT', 'WEETHUSDT', 'STETHUSDT', 'USDSUSDT']
symbols = [item for item in symbols if item not in remove_list]
symbols.sort()
print(symbols)

['0X0USDT', '1INCHUSDT', 'A8USDT', 'AARKUSDT', 'AAVEUSDT', 'ABUSDT', 'ACAUSDT', 'ACEUSDT', 'ACHUSDT', 'ACMUSDT', 'ACTUSDT', 'ACXUSDT', 'ADAUSDT', 'AERGOUSDT', 'AEROUSDT', 'AEVOUSDT', 'AFCUSDT', 'AGLDUSDT', 'AI16ZUSDT', 'AIDOGEUSDT', 'AINUSDT', 'AITECHUSDT', 'AIUSDT', 'AIXBTUSDT', 'ALCHUSDT', 'ALEUSDT', 'ALEXUSDT', 'ALGOUSDT', 'ALICEUSDT', 'ALPACAUSDT', 'ALPHAUSDT', 'ALPHUSDT', 'ALPINEUSDT', 'ALTUSDT', 'ALUSDT', 'ALUUSDT', 'AMPUSDT', 'ANIMEUSDT', 'ANKRUSDT', 'ANLOGUSDT', 'ANYONEUSDT', 'APEUSDT', 'API3USDT', 'APTUSDT', 'APUUSDT', 'AQTUSDT', 'ARBUSDT', 'ARCAUSDT', 'ARCUSDT', 'ARGUSDT', 'ARKMUSDT', 'ARKUSDT', 'ARPAUSDT', 'ARUSDT', 'ASIUSDT', 'ASRUSDT', 'ASTOUSDT', 'ASTRUSDT', 'ATHUSDT', 'ATMUSDT', 'ATOMUSDT', 'AUCTIONUSDT', 'AUDIOUSDT', 'AURORAUSDT', 'AVAAIUSDT', 'AVAILUSDT', 'AVAUSDT', 'AVAXUSDT', 'AXLUSDT', 'AXSUSDT', 'AZEROUSDT', 'AZITUSDT', 'BABYDOGEUSDT', 'BABYUSDT', 'BADGERUSDT', 'BADUSDT', 'BAIUSDT', 'BAKEUSDT', 'BALUSDT', 'BANANAS31USDT', 'BANANAUSDT', 'BANDUSDT', 'BANKUSDT', 'BANU

In [8]:
def get_df_from_ticker(ticker_name, granularity, time_from, time_to) :
    ticker = get_ticker(ticker_name + 'USDT', granularity, time_from, time_to)
    if len(ticker['data']) == 0 :
        return None
    df_1 = pd.DataFrame(ticker['data'], columns=[
    'Timestamp',
    'Open',
    'High',
    'Low',
    'Close',
    'Volume Base',
    'Volume USDT',
    'Volume Quote'])
    df_1['Timestamp'] = pd.to_numeric(df_1['Timestamp'])
    df_1['Date'] = pd.to_datetime(df_1['Timestamp'], unit='ms')
    df_1['Open'] = pd.to_numeric(df_1['Open'], errors='coerce')
    df_1['Close'] = pd.to_numeric(df_1['Close'], errors='coerce')
    df_1['High'] = pd.to_numeric(df_1['High'], errors='coerce')
    df_1['Low'] = pd.to_numeric(df_1['Low'], errors='coerce')
    df_1['Volume'] = pd.to_numeric(df_1['Volume USDT'], errors='coerce')
    df_1 = df_1.sort_values(by='Date')
    return df_1

In [9]:
df_btc_usdt = get_df_from_ticker('BTC', time_frame, time_from_ms, time_to_ms)

In [10]:
def get_history_btc_pair(ticker_name, df_coin_usdt, df_btc_usdt_local) :
    ticker_name_low = ticker_name.lower()
    df_merged = pd.merge(df_coin_usdt, df_btc_usdt_local, on='Date', suffixes=('_' + ticker_name_low, '_btc'))
    df_merged['Open_' + ticker_name_low +'_btc'] = df_merged['Open_' + ticker_name_low] / df_merged['Open_btc']
    df_merged['High_' + ticker_name_low +'_btc'] = df_merged['High_' + ticker_name_low] / df_merged['High_btc']
    df_merged['Low_' + ticker_name_low + '_btc'] = df_merged['Low_' + ticker_name_low] / df_merged['Low_btc']
    df_merged['Close_'+ ticker_name_low +'_btc'] = df_merged['Close_' + ticker_name_low] / df_merged['Close_btc']
    df_merged['Volume_'+ ticker_name_low +'_btc'] = df_merged['Volume_' + ticker_name_low]

    df_final = df_merged[['Date', 'Open_' + ticker_name_low +'_btc', 'High_' + ticker_name_low +'_btc', 'Low_' + ticker_name_low + '_btc', 'Close_' + ticker_name_low +'_btc', 'Volume_' + ticker_name_low +'_btc']].copy()
    df_final.rename(columns={
        'Open_' + ticker_name_low +'_btc': 'Open',
        'High_' + ticker_name_low +'_btc': 'High',
        'Low_' + ticker_name_low + '_btc': 'Low',
        'Close_' + ticker_name_low +'_btc': 'Close',
        'Volume_' + ticker_name_low +'_btc': 'Volume'
    }, inplace=True)

    df_final["Return"] = df_final["Close"].pct_change()

    df_final['RSI_14'] = ta.RSI(df_final['Close'], timeperiod=14)

    df_final['OBV'] = ta.OBV(df_final['Close'], df_final['Volume'])

    slowk, slowd = ta.STOCH(df_final['High'], df_final['Low'], df_final['Close'],
                           fastk_period=5,
                           slowk_period=3,
                           slowd_period=3)
    df_final['Stoch_%K'] = slowk
    df_final['Stoch_%D'] = slowd

    len_data = len(df_merged['Close_' + ticker_name_low])
    if len_data > 2 :
        min_time_period = min(len_data-1, 90)
        df_final['Beta'] = ta.BETA(df_merged['Close_' + ticker_name_low], df_merged['Close_btc'], timeperiod=min_time_period)
        df_final['Correl'] = ta.CORREL(df_merged['Close_' + ticker_name_low], df_merged['Close_btc'], timeperiod=min_time_period)

    df_final['Change1D'] = df_final['Close'].pct_change() * 100
    df_final['Change7D'] = df_final['Close'].pct_change(periods=7) * 100
    df_final['Change30D'] = df_final['Close'].pct_change(periods=30) * 100

    # df_final['EMA_13'] = df_final['Close'].ewm(span=13, adjust=False).mean()
    # df_final['EMA_21'] = df_final['Close'].ewm(span=21, adjust=False).mean()

    return df_final

In [11]:
def show_candlestick(tickername, df) :
    df.set_index('Date', inplace=True)
    mpf.plot(df, type='candle', style='binance', volume=True, title=f'Pergerakan Harga {tickername} terhadap BTC',
         savefig=dict(fname=os.path.join(directory, 'Candlestick-' + tickername + '-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight'))

In [12]:
def show_chart(df, tickername) :
    time_now = datetime.now().strftime('%d-%m-%Y %H:%M:%S')

    # Membuat figure dan dua subplot yang berbagi sumbu x
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=4, ncols=1, sharex=True, figsize=(14, 10))
    # fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(14, 10))

    # Pergerakan Harga
    ax1.plot(df['Date'], df['Close'], label=f'{tickername}/BTC', color='blue')
    # ax1.plot(df['Date'], df['EMA_13'], linewidth=0.5, label='EMA 13', color='yellow')
    # ax1.plot(df['Date'], df['EMA_21'], linewidth=0.5, label='EMA 21', color='orange')
    ax1.set_title(f'Pergerakan Harga {tickername} terhadap BTC')
    ax1.set_ylabel('Harga (BTC)')
    ax1.legend()
    ax1.grid(True)
    ax1.text(0.99, 1.05, time_now,
         transform=ax1.transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    ax1.text(0.5, 0.5, 'Generated by Yoga',
         transform=ax1.transAxes,
         fontsize=20, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)

    # RSI
    ax2.plot(df['Date'], df['RSI_14'], label=f'RSI {tickername}/BTC', color='orange')
    ax2.axhline(70, linestyle='--', alpha=0.5, color='red')  # Garis overbought
    ax2.axhline(30, linestyle='--', alpha=0.5, color='green')  # Garis oversold
    ax2.set_title(f'Grafik RSI {tickername} terhadap BTC')
    ax2.set_xlabel('Tanggal')
    ax2.set_ylabel('RSI')
    ax2.legend()
    ax2.grid(True)
    ax2.text(0.99, 1.05, time_now,
         transform=ax2.transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    ax2.text(0.5, 0.5, 'Generated by Yoga',
         transform=ax2.transAxes,
         fontsize=20, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)

    # Stochastic
    ax3.plot(df['Date'], df['Stoch_%K'], label='%K', color='blue')
    ax3.plot(df['Date'], df['Stoch_%D'], label='%D', color='orange')
    ax3.axhline(80, linestyle='--', alpha=0.5, color='red')  # Garis overbought
    ax3.axhline(20, linestyle='--', alpha=0.5, color='green')  # Garis oversold
    ax3.set_title(f'Grafik Stochastic {tickername} terhadap BTC')
    ax3.set_xlabel('Tanggal')
    ax3.set_ylabel('Nilai')
    ax3.legend()
    ax3.grid(True)
    ax3.text(0.99, 1.05, time_now,
         transform=ax3.transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    ax3.text(0.5, 0.5, 'Generated by Yoga',
         transform=ax3.transAxes,
         fontsize=20, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)

    # OBV
    ax4.plot(df['Date'], df['OBV'], label='OBV', color='blue')
    ax4.set_title(f'Grafik OBV {tickername}')
    ax4.set_xlabel('Tanggal')
    ax4.set_ylabel('Nilai')
    ax4.legend()
    ax4.grid(True)
    ax4.text(0.99, 1.05, time_now,
         transform=ax4.transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    ax4.text(0.5, 0.5, 'Generated by Yoga',
         transform=ax4.transAxes,
         fontsize=20, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)

    # Menyesuaikan tata letak agar tidak saling tumpang tindih
    plt.tight_layout()
    # plt.show()
    plt.savefig(os.path.join(directory, 'Chart-' + tickername + '-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

    show_candlestick(tickername, df)

In [13]:
def show_chart_rsi(df) :
    if 'Label' not in df.columns:
        return
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['RSI_14'], color='blue')
    plt.axhline(70, linestyle='--', alpha=0.5, color='red', label='Overbought (70)')
    plt.axhline(30, linestyle='--', alpha=0.5, color='green', label='Oversold (30)')
    plt.title('Nilai RSI Pair BTC')
    plt.xlabel('Ticker')
    plt.ylabel('RSI')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, 'RSI-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [14]:
def show_chart_beta(df) :
    if 'Label' not in df.columns:
        return
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['Beta'], color='blue')
    plt.axhline(0, linestyle='--', alpha=0.5, color='magenta', label='Beta = 0')
    plt.axhline(1, linestyle='--', alpha=0.5, color='cyan', label='Beta = 1')
    plt.title('Nilai Beta (Volatilitas Harga Token Terhadap BTC)')
    plt.xlabel('Ticker')
    plt.ylabel('Beta')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, 'Beta-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [15]:
def show_chart_correl(df) :
    if 'Label' not in df.columns:
        return
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['Correl'], color='blue')
    plt.axhline(0, linestyle='--', alpha=0.5, color='magenta', label='Correlation = 0')
    plt.axhline(-1, linestyle='--', alpha=0.5, color='cyan', label='Correlation = -1')
    plt.axhline(1, linestyle='--', alpha=0.5, color='cyan', label='Correlation = 1')
    plt.title('Nilai Pearson Correlation Coefficient Terhadap BTC')
    plt.xlabel('Ticker')
    plt.ylabel('Correlation')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, 'Correlation-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [16]:
def show_chart_change(df, chart_title) :
    if 'Label' not in df.columns:
        return
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['Data'], color='blue')
    plt.axhline(0, linestyle='--', alpha=0.5, color='red', label='Change = 0%')
    plt.title(chart_title)
    plt.xlabel('Ticker')
    plt.ylabel('Perubahan (%)')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, chart_title + '-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [17]:
def show_chart_return_mean(df) :
    if 'Label' not in df.columns:
        return
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['Data'], color='blue')
    plt.axhline(0, linestyle='--', alpha=0.5, color='red', label='0')
    plt.title('Rata-Rata Return Pair BTC')
    plt.xlabel('Ticker')
    plt.ylabel('Return Mean (%)')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, 'Rata-Rata Return-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [18]:
def show_chart_return_std(df) :
    if 'Label' not in df.columns:
        return
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['Data'], color='blue')
    plt.axhline(0, linestyle='--', alpha=0.5, color='red', label='0')
    plt.title('Volatilitas Return Pair BTC')
    plt.xlabel('Ticker')
    plt.ylabel('Tingkat Volatilitas')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, 'Volatilitas-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [19]:
def show_chart_return_mean_per_std(df) :
    if 'Label' not in df.columns:
        return
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['Data'], color='blue')
    plt.axhline(0, linestyle='--', alpha=0.5, color='red', label='0')
    plt.title('Rata-Rata Return Per Volatilitas Pair BTC')
    plt.xlabel('Ticker')
    plt.ylabel('Rata-Rata Return Per Volatilitas')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, 'Rata-Rata Return Per Volatilitas-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [20]:
def show_multiple_chart(tickers, df_btc_usdt_local, show_individual_chart=False) :
    rsi_list = []
    beta_list = []
    correl_list = []
    latest_change_1d_list = []
    latest_change_7d_list = []
    latest_change_30d_list = []
    return_mean_list = []
    return_std_list = []
    return_mean_per_std_list = []
    tickers.sort()

    for i, ticker in enumerate(tickers) :
        print('Processing ' + ticker + '...')
        will_show = False
        description = ''
        df_coin_usdt = get_df_from_ticker(ticker, time_frame, time_from_ms, time_to_ms)
        if df_coin_usdt is None or df_coin_usdt.empty :
            print('Skipped ' + ticker + '...')
            continue
        df_btc_pair = get_history_btc_pair(ticker, df_coin_usdt, df_btc_usdt_local)
        if show_individual_chart :
            show_chart(df_btc_pair, ticker)

        if 'RSI_14' in df_btc_pair.columns :
            latest_rsi = df_btc_pair['RSI_14'].iloc[-1]
            if not pd.isna(latest_rsi) :
                rsi_list.append({'Label': ticker, 'RSI_14': latest_rsi})
            if latest_rsi < 30 :
                will_show = True
                description = 'Low RSI'
            if latest_rsi > 70 :
                will_show = True
                description = 'High RSI'

        if 'Beta' in df_btc_pair.columns :
            latest_beta = df_btc_pair['Beta'].iloc[-1]
            if not pd.isna(latest_beta) :
                beta_list.append({'Label': ticker, 'Beta': latest_beta})
            if latest_beta < 0 :
                will_show = True
                description = 'Negative Beta'

        if 'Correl' in df_btc_pair.columns :
            latest_correl = df_btc_pair['Correl'].iloc[-1]
            if not pd.isna(latest_correl) :
                correl_list.append({'Label': ticker, 'Correl': latest_correl})
            if latest_correl < 0 :
                will_show = True
                description = 'Negative Correlation'

        # if will_show and not show_individual_chart :
            # print(description)
            # show_chart(df_btc_pair, ticker)

        latest_change_1d = df_btc_pair['Change1D'].iloc[-1]
        latest_change_7d = df_btc_pair['Change7D'].iloc[-1]
        latest_change_30d = df_btc_pair['Change30D'].iloc[-1]

        if not pd.isna(latest_change_1d) :
            latest_change_1d_list.append({'Label': ticker, 'Data': latest_change_1d})
        if not pd.isna(latest_change_7d) :
            latest_change_7d_list.append({'Label': ticker, 'Data': latest_change_7d})
        if not pd.isna(latest_change_30d) :
            latest_change_30d_list.append({'Label': ticker, 'Data': latest_change_30d})

        return_mean = df_btc_pair['Return'].mean()
        return_std = df_btc_pair['Return'].std()
        return_mean_per_std = return_mean/return_std

        return_mean_list.append({'Label': ticker, 'Data': return_mean*100})
        return_std_list.append({'Label': ticker, 'Data': return_std})
        return_mean_per_std_list.append({'Label': ticker, 'Data': return_mean_per_std})


    rsi_list = sorted(rsi_list, key=lambda x: x['RSI_14'])
    rsi_df = pd.DataFrame(rsi_list)
    show_chart_rsi(rsi_df)

    beta_list = sorted(beta_list, key=lambda x: x['Beta'])
    beta_df = pd.DataFrame(beta_list)
    show_chart_beta(beta_df)

    correl_list = sorted(correl_list, key=lambda x: x['Correl'])
    corr_df = pd.DataFrame(correl_list)
    show_chart_correl(corr_df)

    latest_change_1d_list = sorted(latest_change_1d_list, key=lambda x: x['Data'])
    latest_change_1d_list_df = pd.DataFrame(latest_change_1d_list)
    show_chart_change(latest_change_1d_list_df, 'Perubahan 1 Hari Terhadap BTC')

    latest_change_7d_list = sorted(latest_change_7d_list, key=lambda x: x['Data'])
    latest_change_7d_list_df = pd.DataFrame(latest_change_7d_list)
    show_chart_change(latest_change_7d_list_df, 'Perubahan 7 Hari Terhadap BTC')

    latest_change_30d_list = sorted(latest_change_30d_list, key=lambda x: x['Data'])
    latest_change_30d_list_df = pd.DataFrame(latest_change_30d_list)
    show_chart_change(latest_change_30d_list_df, 'Perubahan 30 Hari Terhadap BTC')

    return_mean_list = sorted(return_mean_list, key=lambda x: x['Data'])
    return_mean_list_df = pd.DataFrame(return_mean_list)
    show_chart_return_mean(return_mean_list_df)

    return_std_list = sorted(return_std_list, key=lambda x: x['Data'])
    return_std_list_df = pd.DataFrame(return_std_list)
    show_chart_return_std(return_std_list_df)

    return_mean_per_std_list = sorted(return_mean_per_std_list, key=lambda x: x['Data'])
    return_mean_per_std_list_df = pd.DataFrame(return_mean_per_std_list)
    show_chart_return_mean_per_std(return_mean_per_std_list_df)

    excel_file = os.path.join(directory, 'price_bitget-' + get_time_stamp() + '.xlsx')
    with pd.ExcelWriter(excel_file) as writer:
        rsi_df.to_excel(writer, sheet_name='RSI', index=False)
        beta_df.to_excel(writer, sheet_name='Beta', index=False)
        corr_df.to_excel(writer, sheet_name='Correlation', index=False)
        latest_change_1d_list_df.to_excel(writer, sheet_name='Change 1D', index=False)
        latest_change_7d_list_df.to_excel(writer, sheet_name='Change 7D', index=False)
        latest_change_30d_list_df.to_excel(writer, sheet_name='Change 30D', index=False)
        return_mean_list_df.to_excel(writer, sheet_name='Return Mean', index=False)
        return_std_list_df.to_excel(writer, sheet_name='Volatility', index=False)
        return_mean_per_std_list_df.to_excel(writer, sheet_name='Return Mean Per Volatility', index=False)

In [21]:
tickers = [symbol.removesuffix('USDT') for symbol in symbols]

show_multiple_chart(tickers, df_btc_usdt, True)

Processing 0X0...
Processing 1INCH...
Processing A8...
Processing AARK...
Processing AAVE...
Processing AB...
Processing ACA...
Processing ACE...
Processing ACH...
Processing ACM...
Processing ACT...
Processing ACX...
Processing ADA...
Processing AERGO...
Processing AERO...
Processing AEVO...
Processing AFC...
Processing AGLD...
Processing AI...
Processing AI16Z...
Processing AIDOGE...
Processing AIN...
Processing AITECH...
Processing AIXBT...
Processing AL...
Processing ALCH...
Processing ALE...
Processing ALEX...
Processing ALGO...
Processing ALICE...
Processing ALPACA...
Processing ALPH...
Processing ALPHA...
Processing ALPINE...
Processing ALT...
Processing ALU...
Processing AMP...
Processing ANIME...
Processing ANKR...
Processing ANLOG...
Processing ANYONE...
Processing APE...


KeyboardInterrupt: 